In [ ]:
%matplotlib inline
import os
os.environ['PY3_PROD'] = '1'
%load_ext autoreload
%autoreload 2
os.system('kinit')

In [ ]:
import numpy as np
import math
import pandas as pd
from typing import Union, List
import datetime
from pycmqlib3.utility import misc
from pycmqlib3.utility.dbaccess import *
import pycmqlib3.analytics.data_handler as dh
import pathlib
from pycmqlib3.utility.process_wt_data import *
from wtpy.wrapper import WtDataHelper
from pycmqlib3.utility import dataseries
# from pycmqlib3.analytics.tstool import *
# from pycmqlib3.analytics.btmetrics import *


In [ ]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

In [ ]:
from misc_scripts.update_fut_prices import *
from misc_scripts.fun_factor_update import *

In [ ]:
# xdf = load_hist_fut_prices(["TA", "CF", "SM"], start_date=datetime.date(2010,1, 1), end_date=datetime.date(2025,1,17))
# xdf
refresh_saved_fut_prices(run_date=datetime.date(2025,6, 7), last_update=datetime.date(2025, 6, 1), use_marker=False)

In [ ]:
from misc_scripts.fun_factor_update import *
spot_df = get_fun_data(datetime.date(2021,1,1), datetime.date(2025,4,4))

In [ ]:
from misc_scripts.daily_update_job import update_data_from_xl
update_data_from_xl(lookback=18000)

In [ ]:
base_folder = "C:/dev/wtdev/storage/his"

zip_wt_dir(base_folder, 'C:/dev/data/wt_data_250415.zip', cutoff=datetime.date(2025,4,14))


In [ ]:
base_folder = "C:/dev/wtdev/storage/his"
update_folder = "C:/dev/data/his"
cutoff = datetime.date(2025, 6, 1)
update_wt_store(base_folder=base_folder, update_folder = update_folder, cutoff=cutoff)

In [ ]:
from pycmqlib3.utility import sec_bits, misc

In [ ]:
file_setup = {
    # ('ifind_data.xlsx', 'hist'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 2, 7, 8, 9],
    #                                 'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},
    ('ifind_data.xlsx', 'ferrous_w'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 6, 7, 8],
                                    'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},
    ('ifind_data.xlsx', 'base_w'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 6, 7, 8],
                                       'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},
    ('ifind_data.xlsx', 'const_d'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 2, 7, 8, 9],
                                   'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},
    # # special format from here
    # ('ifind_data.xlsx', 'base_d2'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 6, 7, 8],
    #                                 'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},
    # ('ifind_data.xlsx', 'macro_m'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 6, 7, 8],
    #                                 'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},
    # ('ifind_data.xlsx', 'warrant_d'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 6, 7, 8],
    #                                 'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},
    # ('ifind_daily.xlsx', 'petchem_d'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 2, 7, 8, 9],
    #                                 'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},                                        
    # ('ifind_daily.xlsx', 'macro_d'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 6, 7, 8],
    #                                 'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},
    # ('ifind_daily.xlsx', 'ferrous_d'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 6, 7, 8],
    #                                 'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},
    # ('ifind_daily.xlsx', 'base_d'): {'header': [0, 1, 2, 3], 'skiprows': [0, 1, 6, 7, 8],
    #                                  'source': 'ifind', 'reorder': [0, 1, 2, 3], 'drop_zero': False},
}

data_folder=sec_bits.LOCAL_NUTSTORE_FOLDER
lookback=20000
sdate = pd.to_datetime("today") - pd.Timedelta(lookback, unit='D')

error_list = []
for data_file, sheet_name in file_setup:
    key = (data_file, sheet_name)
    xdf = pd.read_excel(f'{data_folder}/{data_file}',
                        sheet_name=sheet_name,
                        header=file_setup[key]['header'],
                        skiprows=file_setup[key]['skiprows'])
    xdf = xdf.reorder_levels(file_setup[key]['reorder'], axis=1)
    xdf.columns = [col if idx > 0 else 'date' for idx, col in enumerate(xdf.columns)]
    xdf = xdf.set_index('date')
    if file_setup[key]['drop_zero']:
        xdf = xdf.replace(0, np.nan)
    xdf = xdf[xdf.index >= sdate]
    print(f"saving data for {data_file}:{sheet_name}, total cols:{len(xdf.columns)}, col0={xdf.columns[0]}")

In [ ]:
# merge/fix the data

from pycmqlib3.utility.process_wt_data import *
src_folder  = "D:/data/his"
dst_folder = "C:/dev/wtdev/storage/his"
target_folder  = "D:/data/his_combined"
combine_bars_wt_store(src_folder, dst_folder, target_folder, cutoff=20240928)

In [ ]:
dtHelper = WtDataHelper()
period = 'min5'
exch='CZCE'
cont = 'FG509'
#src_folder = "C:/dev/wtdev/storage/his"
src_folder = "C:/dev/data/his"
#src_folder = "G:\storage_backup\his"
src_path = '%s/%s/%s' % (src_folder, period, exch)
file=f"{cont}.dsb"
src_df = dtHelper.read_dsb_bars(f'{src_path}/{file}')
#src_df = src_df.to_df().rename(columns={'bartime': 'time', 'volume': 'vol'})
#src_df['time'] = src_df['time']
#src_df = src_df[src_df['vol']>0]
#src_df[src_df["time"]>=202412172000].head(20)
src_df.to_df()

In [ ]:
nb_cont=1
start_date=datetime.date(2014,1,1)
end_date=datetime.date(2025,3,28)
roll_rule='-35b'
rb1_df = nearby('rb', n=nb_cont,
                 start_date=start_date,
                 end_date=end_date,
                 roll_rule=roll_rule, freq='d', shift_mode=1)
rb1_df.index = pd.to_datetime(rb1_df.index)
hc1_df = nearby('hc', n=nb_cont,
                 start_date=start_date,
                 end_date=end_date,
                 roll_rule=roll_rule, freq='d', shift_mode=1)
hc1_df.index = pd.to_datetime(hc1_df.index)
rb_hc_spd = rb1_df['close'] - hc1_df['close']

In [ ]:
rb_hc_spd.plot()

In [ ]:
from pycmqlib3.utility.process_wt_data import load_fut_by_product, load_bars_to_df, load_bars_by_code

import datetime
sdate = datetime.date(2025,1,20)
edate = datetime.date(2025,1,20)
period = 'm1'
df1 = load_bars_to_df('SHFE.rb.2510', period=period, start_time=sdate, end_time=edate)
df1

In [ ]:
from misc_scripts.fun_factor_update import *

In [ ]:
spot_df = get_fun_data(start_date=datetime.date(2000, 1, 1), run_date=datetime.date(2025, 3, 28))

In [ ]:
data_df = load_codes_from_edb(index_map.keys(), source='ifind', column_name='index_code')

In [ ]:
start_date='2000-01-01'
run_date='2025-03-31'
cdate_rng = pd.date_range(start=start_date, end=run_date, freq='D', name='date')
data_df = data_df.rename(columns=index_map)
spot_df = data_df.dropna(how='all').copy(deep=True)
spot_df = spot_df.reindex(index=cdate_rng)
sdf = adj_publish_time(spot_df)

In [ ]:
#from pycmqlib3.utility.spot_idx_map import 
spot_df['hrc_prod_all'].dropna().tail(30)

In [ ]:
col_list = [
    'rebar_inv_mill', 'wirerod_inv_mill', 'hrc_inv_mill', 'crc_inv_mill', 'plate_inv_mill',
    'rebar_inv_social', 'wirerod_inv_social', 'hrc_inv_social', 'crc_inv_social', 'plate_inv_social',
    'steel_inv_social', 'rebar_inv_all', 'rebar_prod_all', 'wirerod_prod_all', 'wirerod_inv_all',
    'hrc_prod_all', 'hrc_inv_all', 'crc_prod_all', 'crc_inv_all',
]
spot_df[col_list].dropna(how='all').interpolate(method='linear', limit=1, limit_area='inside')

In [ ]:
from misc_scripts.update_fut_prices import refresh_saved_fut_prices
refresh_saved_fut_prices(run_date=datetime.date(2025,1,18))

C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_mdf['shift'] = curr_mdf['shift'] - shift
C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_mdf['shift'] = curr_mdf['shift'] - shift
C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_mdf['shift'] = curr_mdf['shift'] - shift
C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_mdf['shift'] = curr_mdf['shift'] - shift
C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_mdf['shift'] = curr_mdf['shift'] - shift
C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_mdf['shift'] = curr_mdf['shift'] - shift
C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_mdf['shift'] = curr_mdf['shift'] - shift
C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_mdf['shift'] = curr_mdf['shift'] - shift
C:\dev\pyktrader3\misc_scripts\update_fut_prices.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

{'daily_data': {'rbc1':            contract         open         high          low        close  \
  date                                                                      
  2009-03-30   rb0909  2784.139786  2807.667727  2766.885962  2779.434197   
  2009-03-31   rb0909  2774.728609  2796.688021  2769.238756  2783.355521   
  2009-04-01   rb0909  2791.982433  2792.766698  2778.649932  2781.786991   
  2009-04-02   rb0909  2780.218462  2782.571256  2710.418901  2723.751402   
  2009-04-03   rb0909  2730.025519  2730.025519  2694.733607  2704.144783   
  ...             ...          ...          ...          ...          ...   
  2025-01-13   rb2505  3213.000000  3255.000000  3208.000000  3249.000000   
  2025-01-14   rb2505  3249.000000  3293.000000  3240.000000  3283.000000   
  2025-01-15   rb2505  3285.000000  3303.000000  3270.000000  3290.000000   
  2025-01-16   rb2505  3290.000000  3332.000000  3284.000000  3327.000000   
  2025-01-17   rb2505  3330.000000  3378.000000  3316.

In [ ]:
import pickle
data_file="C:/dev/data/cnc_fut_m5_latest.pkl"
with open(data_file, 'rb') as handle:
    df_dict = pickle.load(handle)

In [ ]:
xdf = load_int_stock_daily(["SPY.P", "GDX.P"], dbtable='int_stock_daily', start_date=None, end_date=None)
xdf

In [ ]:
xdf = xdf.sort_index(axis=1)

In [ ]:
xdf

In [ ]:
from pycmqlib3.analytics.backtest_utils import *
spot_df = load_fun_data(datetime.date(2024,8,9))

In [ ]:
from misc_scripts import fun_factor_update

#sgx_daily_update.fetch_fef_3pm_close(cdate=datetime.date(2023,10,6))

#spot_df = fun_factor_update.get_fun_data(datetime.date(2020,1,1), datetime.date(2023,10,6))
spot_df = fun_factor_update.get_fun_data(datetime.date(2020,10,26), datetime.date(2023,11,1))
spot_df

In [ ]:
from misc_scripts.port_position_update import *

update_port_pos(tday=datetime.date(2023,11,2), email_notify=True)

In [ ]:
from pycmqlib3.utility.misc import day_shift, CHN_Holidays, prod2exch, is_workday, \
    nearby, contract_expiry, inst2contmth
fef_list = []
for nb in [2, 3, 4]:
    fef_nb = nearby('FEF', n=nb,
                    start_date=datetime.date(2016, 7, 1),
                    end_date=datetime.date(2024,5,30),
                    roll_rule='-3b', freq='d', shift_mode=2, roll_col='settle')
    fef_nb.loc[fef_nb['settle'] <= 0, 'settle'] = np.nan
    fef_nb.loc[fef_nb['close'] <= 0, 'close'] = np.nan
    fef_list.append(fef_nb['settle'].to_frame(f'FEFc{nb-1}'))
    fef_list.append(fef_nb['close'].to_frame(f'FEFc{nb-1}_close'))
    fef_list.append(fef_nb['shift'].to_frame(f'FEFc{nb-1}_shift'))
fef_data = pd.concat(fef_list, axis=1)
fef_data.index = pd.to_datetime(fef_data.index)  

In [ ]:
import pickle
data_file="C:/dev/data/cnc_fut_prices_latest.pkl"
with open(data_file, 'rb') as handle:
    df_dict = pickle.load(handle)
if 'daily_data' in df_dict:
    daily_dict = df_dict['daily_data']
else:
    daily_dict = {}

if 'min_data' in df_dict:
    min_dict = df_dict['min_data']
else:
    min_dict = {}

In [ ]:
from misc_scripts import update_fut_prices
df1 = update_fut_prices.load_saved_fut(tday=datetime.date(2024,10,26), freq='m', cont='c1', data_file="C:/dev/data/cnc_fut_latest.pkl")
df2 = update_fut_prices.load_saved_fut(tday=datetime.date(2024,10,26), freq='m', cont='c2', data_file="C:/dev/data/cnc_fut_latest.pkl")

In [ ]:
from pycmqlib3.utility import dataseries
from pycmqlib3.utility.misc import day_shift, CHN_Holidays, prod2exch, is_workday, \
    nearby, contract_expiry, inst2contmth

start_date=datetime.date(2018, 1, 1)
end_date=datetime.date(2024, 9, 20)
shift_mode=2
roll_name='hot'
nb_cont=1
freq='m1'

markets = commod_mkts
fields = ['contract', 'open', 'high', 'low', 'close', 'volume', 'openInterest', 'diff_oi', 'expiry', 'mth', 'shift']
data_df = pd.DataFrame()

for prodcode in markets:
    for nb in range(nb_cont):
        xdf = dataseries.nearby(prodcode,
                                nb + 1,
                                start_date=start_date,
                                end_date=end_date,
                                shift_mode=shift_mode,
                                freq=freq,
                                roll_name=roll_name)        
        xdf['expiry'] = xdf['contract'].map(contract_expiry)
        xdf['contmth'] = xdf.apply(lambda x: inst2contmth(x['contract'], x['date']), axis=1)
        xdf['mth'] = xdf['contmth'].apply(lambda x: x // 100 * 12 + x % 100)
        xdf['product'] = prodcode
        xdf['code'] = f'c{nb + 1}'
        data_df = pd.concat([data_df, xdf])


In [ ]:
fields = ['contract', 'open', 'high', 'low', 'close', 'volume', 'openInterest', 'diff_oi', 'expiry', 'mth', 'shift']
pd.pivot_table(data_df, index='datetime', columns=['product', 'code'], values=fields, aggfunc='last')

In [ ]:
from pycmqlib3.strategy.signal_repo import *
from misc_scripts.fun_factor_update import *
from pycmqlib3.strategy.signal_repo import *
from pycmqlib3.utility.spot_idx_map import index_map, process_spot_df
from pycmqlib3.utility.dbaccess import load_codes_from_edb, load_factor_data
from pycmqlib3.utility import dataseries
from pycmqlib3.utility.misc import day_shift, CHN_Holidays, prod2exch, is_workday, \
    nearby, contract_expiry, inst2contmth
from pycmqlib3.analytics.tstool import *
from misc_scripts.factor_data_update import update_factor_db

run_date = datetime.date(2024,11,4)
start_date = day_shift(run_date, '-4y')
update_start = day_shift(run_date, '-120b', CHN_Holidays)
markets = ['cu', 'al', 'zn', 'pb', 'ni', 'ss', 'sn', 'ao', 'si',
           'rb', 'hc', 'i', 'j', 'jm', 'SM', 'SF', 'v', 'FG', 'SA']
price_df = load_hist_fut_prices(markets, start_date=start_date, end_date=run_date)
cutoff_date = day_shift(day_shift(run_date, '1b', CHN_Holidays), '-1d')
spot_df = get_fun_data(start_date, run_date)

C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connectio

C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connectio

C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connectio

C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connectio

C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql(stmt, cnx)
C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:679: UserWarning: pandas only supports SQLAlchemy connectable (engine/connectio

In [ ]:
start_date = datetime.date(2022,12,22)
end_date = datetime.date(2022,12,23)
src_folder = "C:/dev/wtdev/storage/his"
dst_folder = "C:/dev/wtdev/storage/his"
cutoff_date = 20221223

save_bars_to_wt_store(exchange_list=['GFEX'],
                        start_date=start_date,
                        end_date=end_date,
                        cutoff_date=cutoff_date,
                        process_min=True,
                        process_day=True,
                        src_folder=src_folder,
                        dst_folder=dst_folder)

In [ ]:
start_date = datetime.date(2022,12,22)
end_date = datetime.date(2022,12,22)
dst_folder = "C:/dev/wtdev/storage/his"

save_ticks_to_wt_store(
        exchange_list=['GFEX'],
        start_date=start_date,
        end_date=end_date,
        dst_folder=dst_folder,
        exclusion_list=[])

In [ ]:
from pycmqlib3.utility.dataseries import *
nearby('lu', n=2, start_date=datetime.date(2022,1,1), end_date=datetime.date(2023,10,9), 
       freq='d', shift_mode=0, roll_name='hot', config_loc="C:/dev/wtdev/config/")


In [ ]:
period = 'day'
exch = 'DCE'
cont = 'i2505'
src_folder = 'c:/dev/wtdev/storage/his'
filename = '%s/%s/%s/%s.dsb' % (src_folder, period, exch, cont)
dtHelper = WtDataHelper()
curr_df = dtHelper.read_dsb_bars(filename).to_df()
curr_df

In [ ]:
period = 'day'
exch = 'SHFE'
cont = 'hc2501'
src_folder = 'c:/dev/wtdev/storage/his'
datestr = '20241205'
filename = '%s/ticks/%s/%s/%s.dsb' % (src_folder, exch, datestr, cont)
dtHelper = WtDataHelper()
curr_df = dtHelper.read_dsb_ticks(filename).to_df()
curr_df.tail(20)

In [ ]:
sdate = datetime.date(2000,1,1)
edate = datetime.date(2022,9,30)

cnx = connect(**dbconfig)
cont = 'TA301'
mdf = load_min_data_to_df(cnx, 'fut_min', cont, sdate, edate, index_col = None, shift_datetime = True)

# mdf['rdate'] = mdf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
# mdf['time'] = 0
# mdf['hold'] = mdf['openInterest']
# mdf['diff'] = mdf['hold'].diff().fillna(0)
# mdf['money'] = mdf['close'] * mdf['volume']
# mdf['vol'] = mdf['volume']
col_list = ['date', 'time', 'open', 'high', 'low', 'close', 'money', 'vol', 'hold', 'diff']
# mdf = mdf[col_list]            
mdf = conv_min_data(mdf)
print(mdf[col_list])

In [ ]:
sdate = datetime.date(2000,1,1)
edate = datetime.date(2022,9,30)

cnx = connect(**dbconfig)
cont = 'TA301'
mdf = load_min_data_to_df(cnx, 'fut_min', cont, sdate, edate, index_col = None, shift_datetime = True)

# mdf['rdate'] = mdf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
# mdf['time'] = 0
# mdf['hold'] = mdf['openInterest']
# mdf['diff'] = mdf['hold'].diff().fillna(0)
# mdf['money'] = mdf['close'] * mdf['volume']
# mdf['vol'] = mdf['volume']
col_list = ['date', 'time', 'open', 'high', 'low', 'close', 'money', 'vol', 'hold', 'diff']
# mdf = mdf[col_list]            
mdf = conv_min_data(mdf)
print(mdf[col_list])

In [ ]:
ddf = load_daily_data_to_df(cnx, 'fut_daily', cont, datetime.date(2000,1,1), edate, index_col='date', field = 'instID', date_as_str = False)
ddf = ddf.rename(columns = {'openInterest': 'open_interest'}).reset_index()
ddf['pre_settle'] = ddf['settle'].shift(1)
ddf['pre_interest'] = ddf['open_interest'].shift(1)
ddf['pre_close'] = ddf['close'].shift(1)
ddf['date'] = ddf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
ddf = ddf.set_index('date')